Notebook used to test the training of DE models

In [1]:
import datetime
import os
import time

import tensorflow as tf
import uncertainty_wizard as uwiz
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

from config import Config
from self_driving_car_batch_generator import Generator
from utils import get_driving_styles
from utils import INPUT_SHAPE
from utils_models import *
from self_driving_car_train import *

np.random.seed(0)

In [2]:
cfg = Config()
cfg.from_pyfile("config_my.py")

True

In [3]:
x_train, x_test, y_train, y_test = load_data(cfg)
x_train, y_train = shuffle(x_train, y_train, random_state=0)
x_test, y_test = shuffle(x_test, y_test, random_state=0)

Loading training set track1['normal', 'recovery', 'reverse']
Loading training set completed in 0:00:00.
Data set: 12251 elements
Training set: 9800 elements
Test set: 2451 elements


In [4]:
def create_dave2_de_model(model_id: int):
        """
        original NVIDIA model w/out Dropout layers
        """
        inputs = keras.Input(shape=INPUT_SHAPE)
        lambda_layer = keras.layers.Lambda(lambda x: x / 127.5 - 1.0, name="lambda_layer")(inputs)
        x = keras.layers.Conv2D(24, (5, 5), activation='relu', strides=(2, 2), kernel_regularizer=l2(1.0e-6))(
            lambda_layer)
        x = keras.layers.Conv2D(36, (5, 5), activation='relu', strides=(2, 2), kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Conv2D(48, (5, 5), activation='relu', strides=(2, 2), kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(100, activation='relu', kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Dense(50, activation='relu', kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Dense(10, activation='relu', kernel_regularizer=l2(1.0e-6))(x)
        outputs = keras.layers.Dense(1)(x)
        model = keras.Model(inputs=inputs, outputs=outputs)

        model.compile(loss='mean_squared_error', optimizer=Adam(lr=cfg.LEARNING_RATE))


        train_generator = Generator(x_train, y_train, True, cfg)
        val_generator = Generator(x_test, y_test, False, cfg)

        history = model.fit(train_generator,
                                validation_data=val_generator,
                                batch_size=128,
                                epochs=1,
                                callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)],
                                verbose=1)

        return model, history.history


In [5]:
def create_de(model_id: int):
    """
    Load train/validation data_nominal set and train the model
    """
    cfg = Config()
    cfg.from_pyfile("config_my.py")

    x_train, x_test, y_train, y_test = load_data(cfg)


    model = build_model(cfg.SDC_MODEL_NAME, cfg.DROPOUT_RATE,cfg.USE_PREDICTIVE_UNCERTAINTY)
    train_model(model, cfg, x_train, x_test, y_train, y_test)

    return model

In [7]:
ensemble = uwiz.models.LazyEnsemble(num_models=5, # For the sake of this example. Use more in practice!
                                    model_save_path="models\DE",
                                    # Colab infrastructure is relatively weak.
                                    # Thus, lets disable multiprocessing and train on the main process.
                                    # Any argument >= 1 would result in (typically more efficient) multiprocessing
                                    # on a more powerful machine
                                    default_num_processes=0)
# Creates, trains and persists atomic models using our function defined above
training_histories = ensemble.create(create_function=create_dave2_de_model)

Working on model 0 (of 5) in the main process
38/38 [==============================] - 36s 942ms/step - loss: nan - val_loss: nan


c:\Users\ruben\anaconda3\envs\udacity-self-driving-car\lib\site-packages\tensorflow\python\keras\callbacks.py:1664: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current - self.min_delta, self.best):


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: c:\Users\ruben\Documents\Ruben\TUM\Thesis\Workspaces\white-box-ads-23\models\DE\0\assets
Working on model 1 (of 5) in the main process
38/38 [==============================] - 36s 938ms/step - loss: 0.0404 - val_loss: 0.0627
INFO:tensorflow:Assets written to: c:\Users\ruben\Documents\Ruben\TUM\Thesis\Workspaces\white-box-ads-23\models\DE\1\assets
Working on model 2 (of 5) in the main process
38/38 [==============================] - 36s 954ms/step - loss: 0.0403 - val_loss: 0.1944
INFO:tensorflow:Assets written to: c:\Users\ruben\Documents\Ruben\TUM\Thesis\Workspaces\white-box-ads-23\models\DE\2\assets
Working on model 3 (of 5) in the main process
38/38 [==============================] - 35s 928ms/step - loss: 0.0399 - val

In [8]:
val_generator = Generator(x_test, y_test, False, cfg)
predictions, confidences = ensemble.predict_quantified(val_generator,
                                                       quantifier='standard_deviation')

Working on model 0 (of 10) in the main process
8/8 [==============================] - 2s 188ms/step
Working on model 1 (of 10) in the main process
8/8 [==============================] - 1s 179ms/step
Working on model 2 (of 10) in the main process
8/8 [==============================] - 1s 182ms/step
Working on model 3 (of 10) in the main process
8/8 [==============================] - 1s 177ms/step
Working on model 4 (of 10) in the main process
8/8 [==============================] - 1s 180ms/step
Working on model 5 (of 10) in the main process
8/8 [==============================] - 1s 179ms/step
Working on model 6 (of 10) in the main process
8/8 [==============================] - 1s 184ms/step
Working on model 7 (of 10) in the main process
8/8 [==============================] - 2s 188ms/step
Working on model 8 (of 10) in the main process
8/8 [==============================] - 1s 187ms/step
Working on model 9 (of 10) in the main process
8/8 [==============================] - 1s 184ms/step


In [9]:
predictions

array([[-0.05994238],
       [-0.05059595],
       [-0.05504868],
       ...,
       [ 0.0028793 ],
       [ 0.03574293],
       [ 0.10860348]])

In [10]:
confidences

array([[0.01295222],
       [0.01843196],
       [0.01637283],
       ...,
       [0.01801964],
       [0.01849108],
       [0.03789418]])

In [13]:
predictions.shape

(1024, 1)